## Import the Libs

In [ ]:
import sys 
import os
import matplotlib.pyplot as plt
try:
    sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname(__file__), '..')))
except:
    sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
import torch
from Methods.TransferMatrixMethod.Wavelength import WaveLength
from Methods.TransferMatrixMethod.Structure  import Structure
from Methods.TransferMatrixMethod.Layer      import Layer
from Materials.Materials                     import Material 
from Materials.Materials                     import CompositeMaterial
from Methods.TransferMatrixMethod.AnnularPhotonicTransferMatrix import AnnularPhotonicTransferMatrix
from Materials.Materials_database.Materials.MaterialsFunctions.PVA import refractive_index_PVA
from Study.OutputAnalysis                    import Analysis
from Study.Study                             import StudySwipe
from Study.Study                             import SaveData, LoadData
from Methods.PhysicalQuantity                import PhysicalQuantity
from Visualization.Visualization             import visualization
from Visualization.Visualization             import plot
from Utils.Utils                             import batched_values
from Utils.Export                            import ExportUtils
import traceback

In [ ]:
%load_ext autoreload
%autoreload 2

# from Experiments.GammaRay import material_sensor, get_parameters

# parameters, parameters_analysis, experiment_path, experiment_name = get_parameters()
# print(experiment_path)

## Plot the refractive index

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_y(x, y, labels, x_label, y_label, save_path, before_after_label):
    print("Shape of " + y_label + ": ", y.shape)
    colors = ['blue', 'red', 'green', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']
    lines = ['-', '--', '-.', ':']
    # symobls = ['o', 's', 'D', 'v', '^', '<', '>', 'p', 'h', 'H', 'd', 'P', 'X']
    fig, axes = plt.subplots(1, 1, figsize=(7, 5))
    for i, label in enumerate(labels):
        axes.plot(x, y[i], 
                label=before_after_label[0]+str(label)+before_after_label[1] if label is not None else None,
                linestyle=lines[i%4],
                # marker=symobls[i%13],
                color=colors[i])
    axes.legend(loc='best')
    axes.grid()
    axes.set_xlabel( x_label, fontsize=13)
    axes.set_ylabel( y_label, fontsize=13)
    # axes.set_ylim([228,232])
    plt.savefig(save_path, dpi=600)



def plot_y_ploty(x, y, labels=[None], x_label=None, y_label=None, save_path=None, before_after_label=["", ""]):
    print("Shape of " + str(y_label) + ": ", y.shape)

    fig = make_subplots(rows=1, cols=1)

    for i, label in enumerate(labels):
        fig.add_trace(
            go.Scatter(
                x=x,
                y=y[i] if len(labels) > 1 else y,
                name=before_after_label[0] + str(label) + before_after_label[1] if label is not None else None,
                mode='lines'
            )
        )

    fig.update_layout(
        xaxis_title=x_label,
        yaxis_title=y_label,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        ),
        xaxis=dict(title_font=dict(size=13)),
        yaxis=dict(title_font=dict(size=13)),
        width=700,
        height=500
    )

    fig.update_xaxes(showgrid=True)
    fig.update_yaxes(showgrid=True)

    if save_path is not None:
        fig.write_image(save_path, scale=3)  # scale=3 for higher resolution (similar to dpi=600)
        fig.write_html(save_path+".html")

    fig.show()
def plot_matplotlib_plotly(x, y, labels, x_label, y_label, save_path, before_after_label):
    plot_y(x, y, labels, x_label, y_label, save_path, before_after_label)
    save_path = None
    plot_y_ploty(x, y, labels, x_label, y_label, save_path, before_after_label)


In [ ]:
gamma_ray_doses = [0, 10, 20, 30, 40, 50, 60, 70] #Gy
wavelength = WaveLength(ranges=[400,700],steps=0.02, units="m", unit_prefix="n")
refractive_index_variable_material= refractive_index_PVA(wavelength.values*1e9, gamma_ray_doses)
experiment_path = os.path.join(os.getcwd(), "Experiments", "GammaRay")
os.makedirs(experiment_path, exist_ok=True)

plot_matplotlib_plotly(x=wavelength.values*1e9, y=refractive_index_variable_material,
    x_label="Wavelength (nm)", y_label="Polymer refractive index", labels=gamma_ray_doses,
    save_path=os.path.join(experiment_path, "refractive_index_PVA.png"),
    before_after_label=['Dose = ', ' Gy'])

variableProperty = [0, 10, 20, 30, 40, 50, 60, 70] #Gy # Gamma ray dose
wavelength = WaveLength(ranges=[400,750],steps=0.02, units="m", unit_prefix="n")
refractive_index_variable_material= refractive_index_PVA(wavelength.values*1e9, variableProperty)
wavelength.broadcast([len(variableProperty), *wavelength.values.shape])

variableMaterial   = Material(wavelength, name="PVA", refractive_index=refractive_index_variable_material)
Si                 = Material(wavelength, name="Si",   refractive_index=3.7)
SiO2               = Material(wavelength, name="SiO2", refractive_index=1.45)

porosity_1, porosity_2 =     torch.as_tensor(0.7), torch.as_tensor(0.7) 
material_porose_1 = CompositeMaterial.get_material('PorousMedium', [Si,variableMaterial], porosity=porosity_1)
material_porose_2 = CompositeMaterial.get_material('PorousMedium', [SiO2,variableMaterial], porosity=porosity_2)
material_porose_3 = variableMaterial
print("Average refractive index layer 1 is :", material_porose_1.refractive_index_avarage )
print("Average refractive index layer 2 is :", material_porose_2.refractive_index_avarage )
print("Average refractive index of variable material is :", variableMaterial.refractive_index_avarage )


functions = [plot_y]#, plot_y_ploty]

for fun in functions:
    fun(x=wavelength.values[0,...]*1e9, y=material_porose_1.refractive_index,
    x_label="Wavelength (nm)",
    y_label="Refractive index layer a",
    labels=variableProperty,
    save_path=os.path.join(experiment_path, "refractive_index_layer_a.png"),
    before_after_label=['Dose = ', ' Gy'])

    fun(x=wavelength.values[0,...]*1e9, y=material_porose_2.refractive_index,
    x_label="Wavelength (nm)",
    y_label="Refractive index layer b",
    labels=variableProperty,
    save_path=os.path.join(experiment_path, "refractive_index_layer_b.png"),
    before_after_label=['Dose = ', ' Gy'])

    fun(x=wavelength.values[0,...]*1e9, y=material_porose_3.refractive_index,
    x_label="Wavelength (nm)",
    y_label="Refractive index layer c",
    labels=variableProperty,
    save_path=os.path.join(experiment_path, "refractive_index_layer_c.png"),
    before_after_label=['Dose = ', ' Gy'])
# Average refractive index layer 1 is : tensor(3.2405)
# Average refractive index layer 2 is : tensor(2.0966)
        

In [ ]:
# thickness of the layers
d1 = 550/(4*2.7) #nm
d2 = 550/(1.45*4) #nm
df = 550/(4*2.8) #nm
print('d1', d1, 'd2', d2, 'df', df)

## Get the reflectance 

In [ ]:

def get_variable_material(wavelength, variableProperty, batch_size, **kwargs):

    Water        = Material(wavelength, name="Water", refractive_index=1.33230545)
    E_coli       = Material(wavelength, name="E_coli", refractive_index=1.39)
    variable_material = CompositeMaterial.get_material('Linear', [E_coli,Water], Volume_fraction = variableProperty)
    refractive_index_variable_material =  variable_material.refractive_index
    # refractive_index_variable_material = refractive_index_variable_material.unsqueeze(0).repeat(batch_size,1,1)
    variableMaterial                   = Material(wavelength, name="E_coli", refractive_index=refractive_index_variable_material)
    property_value                     = PhysicalQuantity(values=refractive_index_variable_material, units='RIU', name='Refractive index')
    return variableMaterial, property_value

def material_sensor(wavelength, method,
                            N, rho0,
                            layer_1_thickness, layer_2_thickness, 
                            defect_thickness,
                            porosity_1, porosity_2,
                            variableProperty,
                            batch_size,
                            **kwargs):
    
    # get the variable material
    variableMaterial, _ = get_variable_material(wavelength, variableProperty, batch_size)

    # broadcast the values
    wavelength.broadcast([batch_size,len(variableProperty.values), *wavelength.values.shape])
    
    # Define the materials
    Si  = Material(wavelength, name="Si", refractive_index=3.5)
    SiO2 = Material(wavelength, name="SiO2", refractive_index=1.45)
    # Air          = Material(wavelength, name="Air", refractive_index=1)
    # Water        = Material(wavelength, name="Water", refractive_index=1.33230545)
    # E_coli       = Material(wavelength, name="E_coli", refractive_index=1.39)

    material_porose_1 = CompositeMaterial.get_material('PorousMedium', [Si, variableMaterial], porosity=porosity_1)
    material_porose_2 = CompositeMaterial.get_material('PorousMedium', [Si, variableMaterial], porosity=porosity_2)

    # Define the layers
    layer0 = Layer(SiO2,thickness=rho0)
    layerd = Layer(variableMaterial, thickness=defect_thickness)
    layer1 = Layer(material_porose_1, thickness=layer_1_thickness)
    layer2 = Layer(material_porose_2, thickness=layer_2_thickness)
    layerf = Layer(SiO2, thickness=1000e-9)
    # Define the structure
    structure = Structure([layer0, [layer1, layer2], layerd ,[layer1, layer2],layerf],
                                        layers_parameters = {'method':'repeat', 
                                                                'layers_repeat':[1,N,1,N,1],
                                                                })

    # get the reflectance
    outputs = {}
    outputs['Reflectance']  = PhysicalQuantity(values = method.Reflectance_from_layers(structure.layers, m=0, mode= 'TE'), units='a.u.', name='Reflectance')

    return outputs

def analysis(wavelength,
            Results=None,           
            experiment_path=None, load_name=None, 
            PBG_range = None,
            variableProperty = None,
            working_dim = None,referance_index = None,
            batch_size = 1,
            backend='scipy',
            strcuture_type = 'periodic_with_defect', number_of_defect_peaks = 1,
            **kwargs):

    analysis = Analysis()

    if Results is not None:
        R = Results['Reflectance']
    elif experiment_path is not None:
        data = LoadData.get_reference_data(experiment_path, load_name,)
        keys = list(data.keys())
        for key in keys:
            if 'Reflectance' in key:
                R = PhysicalQuantity(values=data[key], units='a.u.', name='Reflectance')
    else:
        raise ValueError('Results and save_dir cannot be None at the same time')

    _, property_value = get_variable_material(wavelength, variableProperty, batch_size)

    wavelength.set_values_in_unit()

    try:
        output = analysis.get_output(wavelength=wavelength, R=R, PBG_range=PBG_range, backend=backend, strcuture_type=strcuture_type, number_of_defect_peaks=number_of_defect_peaks, property_value=property_value, working_dim=working_dim, referance_index=referance_index)
        return output
    except Exception as e:
        print('Error in analysis:', e)
        traceback.print_exc()
        return None

def get_parameters():
    # Define the experiment discription
    try :
        experiment_path = os.path.dirname(__file__)
        experiment_name = os.path.basename(__file__).split('.')[0]
    except Exception as e:
        experiment_path = os.path.join(os.getcwd())
        experiment_name = 'E_coli'

    experiment_path, experiment_name = SaveData.create_experiment_description(experiment_path=experiment_path, experiment_name=experiment_name)
    # Define the method
    method = AnnularPhotonicTransferMatrix()
    batch_size = 1
    wavelength = WaveLength(ranges=[570,620],steps=0.005, units="m", unit_prefix="n")
    # wavelength = WaveLength(ranges=[400,800],steps=5, units="m", unit_prefix="n")

    N                 =  10
    rho0              =  500 * 1e-9 # m
    porosity_1        =  torch.tensor(0.7)
    porosity_2        =  torch.tensor(0.5) #0.4
    layer_1_thickness = 70   * 1e-9 # m  50   * 1e-9 # m  
    layer_2_thickness = 75   * 1e-9 # m  65   * 1e-9 # m  
    defect_thickness  = 70  * 1e-9 # m 48   * 1e-9 # m
    variableProperty   = PhysicalQuantity(values=torch.tensor([0.0,0.2]), units=None, name='Volume fraction')

    parameters = {
        'wavelength':wavelength, 'experiment_path':experiment_path, 'method':method,
        'N':N, 'rho0':rho0,
        'layer_1_thickness':layer_1_thickness, 'layer_2_thickness':layer_2_thickness,
        'defect_thickness':defect_thickness,
        'porosity_1':porosity_1,'porosity_2':porosity_2,
        'variableProperty':variableProperty, 
        'batch_size':batch_size,
        }
    
    parameters_analysis = {
        'wavelength':wavelength, 'experiment_path':experiment_path,
        # 'PBG_range':[420,578],
        'PBG_range':None,
        'variableProperty':variableProperty,  
        'working_dim':-1, 'referance_index':0,
        'backend':'scipy', 'strcuture_type':'periodic_with_defect', 'number_of_defect_peaks':1,
        # 'backend':'scipy', 'strcuture_type':'periodic', 'number_of_defect_peaks':0,
        }
    return parameters, parameters_analysis, experiment_path, experiment_name

if __name__ == "__main__":

    parameters, parameters_analysis, experiment_path, experiment_name = get_parameters()
    # outputs = material_sensor(**parameters)
    # plot_y_ploty(x=parameters['wavelength'].values[0,0,...].squeeze().cpu().numpy()*1e9, y=outputs['Reflectance'].values.squeeze().cpu().numpy(),labels=parameters['variableProperty'].values.squeeze().cpu().numpy(),x_label='Wavelength (nm)',y_label='Reflectance',save_path=None,before_after_label=['Dose = ', ' Gy'])
    # plot_y(x=parameters['wavelength'].values[0,0,...].squeeze().cpu().numpy()*1e9, y=outputs['Reflectance'].values.squeeze().cpu().numpy(),labels=parameters['variableProperty'].values.squeeze().cpu().numpy(),x_label='Wavelength (nm)',y_label='Reflectance',save_path=os.path.join(experiment_path, 'Reflectance' + '.png'),before_after_label=['Volume fraction = ', ''])
    # parameters, parameters_analysis, experiment_path, experiment_name = get_parameters()
    # parameters['variableProperty'] = PhysicalQuantity(values=torch.tensor([0,0.1,0.2,0.3,0.4,0.5]), units=None, name='Volume fraction')
    # outputs = material_sensor(**parameters)
    # data = {"Reflectance_plot1":{'x':parameters['wavelength'].values[0,0,...].squeeze().cpu().numpy()*1e9, 'y': parameters['variableProperty'].values.squeeze().cpu().numpy(),
    #                                 'z':outputs['Reflectance'].values.squeeze().cpu().numpy(),
    #                                 'x_label':'Wavelength (nm)', 'y_label':'Volume fraction', 'z_label':'Reflectance'}}
    # torch.save(data, os.path.join(experiment_path, 'WaveLength_vs_Reflectance' + '.pt'))
    # plot_data_matplotlib(data=data, config=None, experiment_path=experiment_path, data_name='WaveLength_vs_Reflectance', plot_analysis=False )
    


In [ ]:
experiment_path = '/Users/ayman/Documents/photonics/Experiments/E_coli/'
data_path = os.path.join(experiment_path,'data', 'WaveLength_vs_Reflectance' + '.pt')
data = torch.load(data_path)
wavelength = data["Reflectance_plot1"]['x']
variableProperty = data["Reflectance_plot1"]['y']
wavelength = PhysicalQuantity(values=wavelength, units='nm', name='Wavelength')
variableProperty = PhysicalQuantity(values=variableProperty, units=None, name='Volume fraction')
variableMaterial, _ = get_variable_material(wavelength, variableProperty, 1)
refractive_index = [variableMaterial.refractive_index[i][0].item() for i in range(len(variableProperty.values))]
refractive_index = torch.tensor(refractive_index).unsqueeze(0)
plot_y(x=variableProperty.values.squeeze().cpu(), y=refractive_index, labels=[None], x_label='Volume fraction', y_label='Refractive index', save_path=os.path.join(experiment_path, 'refractive_index' + '.png'), before_after_label=['', ''])





In [ ]:
variableMaterial.refractive_index.shape
print(variableProperty.values)
print(refractive_index)
from scipy.optimize import curve_fit
def linear(x, m, c):
    return m*x + c
popt, _ = curve_fit(linear, variableProperty,refractive_index.squeeze())
print(popt)

### Calculate senstivity and fitting it to equation 

In [ ]:

def limit_x_y( x, y, x_value_limit, reversed=False):
    condition = (x>x_value_limit[0]) & (x<x_value_limit[1])
    if reversed:
        condition = ~condition
    x_index = torch.where(condition)
    x = x[x_index]
    y = y[x_index]
    print('x shape:', x.shape, 'y shape:', y.shape)
    return x, y, x_index


def get_defect_peak_and_properiety_Ecoil(data,  plot_word='_plot0', tol=5, x_limit=None, x_limit_correct=None, filled_values = None,):

    R = torch.as_tensor(data['Reflectance'+plot_word]['z'])
    wavelength = torch.as_tensor(data['Reflectance'+plot_word]['x'])
    variable   = torch.as_tensor(data['Reflectance'+plot_word]['y'])
    variableProperty = PhysicalQuantity(values=variable, units=None, name='Volume fraction')
    Wavelength = PhysicalQuantity(values=wavelength, units=None, name='Wavelength')
    refractive_index_variable_material, _  = get_variable_material(Wavelength, variableProperty, batch_size=1)
    refractive_index_variable_material = refractive_index_variable_material.refractive_index.squeeze()
    print(refractive_index_variable_material.shape)
    x_limit = [575,650]
    wavelength, R_PBG, _ = limit_x_y(wavelength,R.T,x_limit)
    R_PBG = R_PBG.T
    print('R_PBG shape:', R_PBG.shape)
    fig = make_subplots(rows=1, cols=1)
    z =R_PBG[:,::5]
    print(z.shape)
    heatmap = go.Heatmap(
        z=z,
        colorscale='Plasma'
    )
    fig.add_trace(heatmap, row=1, col=1)
    fig.show()
    import scipy
    import numpy as np
    wavelength_peak  = np.zeros(R_PBG.shape[0])
    refractive_index_wavelength_peak = np.zeros(R_PBG.shape[0])
    for i in range(R_PBG.shape[0]):
        try:
            peaks, peaks_properties = scipy.signal.find_peaks(1-R_PBG[i].cpu().numpy().squeeze(), prominence = 0.1)
            wavelength_peak[i] = wavelength[peaks[peaks_properties['prominences'].argsort()[-1]]]
            print('wavelength_peak:', wavelength_peak[i])
            refractive_index_wavelength_peak[i] = refractive_index_variable_material[i,torch.where(wavelength==wavelength_peak[i])].item()
        except:
            print('Error in finding the peak')
            wavelength_peak[i] = np.nan
            refractive_index_wavelength_peak[i] = np.nan            
    print(wavelength_peak.shape)
    print(variable.shape)

    return  wavelength_peak, refractive_index_wavelength_peak

experiment_path = '/Users/ayman/Documents/photonics/Experiments/E_coli/'
data_path = os.path.join(experiment_path,'data', 'WaveLength_vs_Reflectance' + '.pt')
data = torch.load(data_path)
plot_words = ['_plot1']
tols = [None]
x_limits = [None]
x_limit_corrects = [None]
for i in range(len(plot_words)):
    wavelength_peak, refractive_index_wavelength_peak = get_defect_peak_and_properiety_Ecoil(data, plot_word=plot_words[i], tol=tols[i], x_limit=x_limits[i], x_limit_correct=x_limit_corrects[i])



In [ ]:
sen = (wavelength_peak[1:] - wavelength_peak[0]) / (refractive_index_wavelength_peak[1:] - refractive_index_wavelength_peak[0])
print(sen)
plot_y(x=[0.1, 0.2, 0.3, 0.4, 0.5],
       y=torch.as_tensor([sen]),
       labels = [None],
       x_label="Volume fraction",
       y_label="Sensitivity (nm/RIU)",
       save_path="Sensitivity.png",
       before_after_label=[","])


In [ ]:
print(data['Reflectance_plot1'].keys())  

## Plotting the sweep parameters

In [ ]:
def set_x_and_y_limits(ax, x=None,y=None, x_range_addition=0.2, y_range_addition=0.1):

    if x is not None and y is not None:

        x,y = torch.as_tensor(x), torch.as_tensor(y)
        y_lim_new =[y.min().item(), y.max().item()]
        # put 10 % margin
        y_lim_new[0] = y_lim_new[0] - y_range_addition*(y_lim_new[1] - y_lim_new[0])
        y_lim_new[1] = y_lim_new[1] + y_range_addition*(y_lim_new[1] - y_lim_new[0])
        

        x_lim_new =[x.min().item(), x.max().item()]
        # put 10 % margin
        x_lim_new[0] = x_lim_new[0] - x_range_addition *(x_lim_new[1] - x_lim_new[0])
        x_lim_new[1] = x_lim_new[1] + x_range_addition *(x_lim_new[1] - x_lim_new[0])

        ax.set_xlim(x_lim_new)
        print('x limit', x_lim_new)
        ax.set_ylim(y_lim_new)
        print('y limit', y_lim_new)


In [ ]:
def prepare_data(data, key1, key2 = None, config = None, round=2):
    def get_data_where(index, x, y, y2=None, z=None, dim_z=0):
        x = x[index]
        y = y[index]
        if y2 is not None:
            y2 = y2[index]
        if z is not None:
            if dim_z == 0:
                z = z[index,:,:]
            elif dim_z == 1:
                z = z[:,index,:]
            else:
                raise ValueError('dim_z should be 0 or 1')
        return x,y,y2,z 
    
    def get_index_del_points(x, del_points, round=2):
        if round is not None:
            x = torch.round(x*10**round)/10**round
            del_points = torch.round(torch.as_tensor(del_points)*10**round)/10**round
        index = torch.where(torch.logical_not(torch.isin(x, torch.as_tensor(del_points))))
        return index
    
    x = data[key1]['x']
    y = data[key1]['y']
    y2 = data[key2]['y'] if key2 is not None else None 
    z = data[key1]['z'] if 'z' in data[key1] else None
    x_label = data[key1]['x_label']
    y_label = data[key1]['y_label']
    z_label = data[key1]['z_label'] if 'z_label' in data[key1] else None
    config = config[key1] if config is not None else None 

    try:
        x_lim = config['xlim']
        y_lim = config['ylim']
    except:
        x_lim = None
        y_lim = None

    
    if x_lim is not None:
        index = torch.where((x>x_lim[0]) & (x<x_lim[1]))
        x,y,y2,z = get_data_where(index, x, y, y2, z, dim_z=0)

    if y_lim is not None:
        index = torch.where((y>y_lim[0]) & (y<y_lim[1]))
        x,y,y2,z = get_data_where(index, x, y, y2, z, dim_z=1)

    try: 
        del_points_x = config['del_points_x']
        del_points_y = config['del_points_y']
    except:
        del_points_x = None
        del_points_y = None
    
    if del_points_x is not None:
        index = get_index_del_points(x, del_points_x, round=round)
        x,y,y2,z = get_data_where(index, x, y, y2, z, dim_z=0)
    if del_points_y is not None:
        index = get_index_del_points(y, del_points_y, round=round)
        x,y,y2,z = get_data_where(index, x, y, y2, z, dim_z=1)

    data[key1]['x'] = x
    data[key1]['y'] = y
    if y2 is not None:
        data[key2]['x'] = x
        data[key2]['y'] = y2
    if z is not None:
        data[key1]['z'] = z

    return data, x,y,y2,z, x_label, y_label, z_label

# def plot_data_matplotlib(data, config, experiment_path, data_name, plot_heatmap = True, plot_analysis = True, labels=None, before_after_label=["", ""]):
#     if plot_heatmap:
#         fig, axes = plt.subplots(1,1, figsize=(7, 5))
#         key = 'Reflectance_plot1'
#         data, x, y, _, z, x_label, y_label, z_label = prepare_data(data, key, key2=None, config=config)
#         heatmap = axes.pcolor(x,y,z, cmap='inferno') # 'hot', 'viridis' , 'inferno' , 'plasma' , 'magma' , 'cividis' , 'twilight' , 'twilight_shifted'
#         axes.set_xlabel(x_label, fontsize=13)
#         axes.set_ylabel(y_label, fontsize=13)
#         fig.colorbar(heatmap, ax=axes, label=z_label)      
#         plt.savefig(experiment_path+'/'+data_name+ "Reflectance1" + '.png', dpi=600)
#         plt.close()

#     if plot_analysis:
#         # Defect 
#         rows, cols = 3, 2
#         keys  = ['peak_wavelength_plot0', 'defect_intensity_R_plot0', 'FullWidthHalfMax_plot0', 'QualityFactor_plot0', 'Sensitivity_plot0', 'FigureOfMerit_plot0']
#         keys2 = ['peak_wavelength_plot1', 'defect_intensity_R_plot1', 'FullWidthHalfMax_plot1', 'QualityFactor_plot1', None, None]
#         rows, cols = 2, 2
#         keys  = ['peak_wavelength_plot0', 'QualityFactor_plot0', 'Sensitivity_plot0', 'FigureOfMerit_plot0']
#         keys2 = ['peak_wavelength_plot1', 'QualityFactor_plot1', None, None]
#         # Periodic
#         # rows, cols = 3, 2
#         # keys  = [ 'Sensitivity_plot0', 'band_gap_begining_plot0', 'band_gap_end_plot0', 'band_gap_width_plot0', 'band_gap_center_plot0', 'average_peak_intensity_plot0', ]
#         # keys2 = [  None, 'band_gap_begining_plot1', 'band_gap_end_plot1', 'band_gap_width_plot1', 'band_gap_center_plot1', 'average_peak_intensity_plot1', ]
#         texts = ['(a)', '(b)', '(c)', '(d)', '(e)', '(f)', '(g)', '(h)', '(i)', '(j)', '(k)', '(l)', '(m)', '(n)', '(o)', '(p)', '(q)', '(r)', '(s)', '(t)', '(u)', '(v)', '(w)', '(x)', '(y)', '(z)']
#         colors = ['blue', 'red', 'green', 'black', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan', 'magenta', 'yellow', 'lime', 'teal', 'coral', 'lightblue', 'lightgreen', 'lightcoral', 'lightpink', 'lightgray', 'lightolive', 'lightcyan', 'lightmagenta', 'lightyellow', 'lightlime', 'lightteal', 'lightcoral']
#         fig, axes = plt.subplots(rows, cols, figsize=(13, 15))
#         for i, key in enumerate(keys):
#             data, x, y, y2, _, x_label, y_label, _ = prepare_data(data, key, key2=keys2[i], config=config)
#             x_index, y_index = i//2, i%2
#             if y is not None:
#                 axes[x_index, y_index].plot(x, y, color=colors[i])
#             if y2 is not None:
#                 axes[x_index, y_index].plot(x, y2, color=colors[i+(rows*cols)], linestyle='--')
#             axes[x_index, y_index].grid()
#             axes[x_index, y_index].set_xlabel(x_label, fontsize=13)
#             axes[x_index, y_index].set_ylabel(y_label, fontsize=13)

#             # y = y if y2 is None else torch.concatenate((y.unsqueeze(-1), y2.unsqueeze(-1)), dim=-1)
#             # set_x_and_y_limits(axes[x_index, y_index], x=x, y=y)
#             if labels is not None and y is not None and y2 is not None:
#                 labels_final = [before_after_label[0]+str(label)+before_after_label[1] for label in labels]
#                 # axes[x_index, y_index].legend(labels_final, loc='best')
#                 axes[x_index, y_index].legend(labels_final, loc='lower left')
            
#             # if key in ['defect_intensity_R_plot0','defect_intensity_R_plot1','average_peak_intensity_plot0']:
#             #     # set_x_and_y_limits(axes[x_index, y_index], x=x, y=y, x_range_addition=0, y_range_addition=10)
#             #     # axes[x_index, y_index].set_ylim([-0.05, 0.05])
#             #     axes[x_index, y_index].set_ylim([0.5, 1.1])

#             # if key in ['Sensitivity_plot0']:
#             #     axes[x_index, y_index].set_ylim([190, 210])
                
#             axes[x_index, y_index].text(0.1, 0.85, texts[i], color='black', backgroundcolor='white', transform=axes[x_index, y_index].transAxes, fontsize=20)
#         # save the figure with very high resolution
#         plt.savefig(experiment_path+'/'+data_name+'_data_.png', dpi=600)

#         plt.show()
#         plt.close()
#     # print(data)
#     return data

def plot_data_matplotlib(data, config, experiment_path, data_name, plot_heatmap = True, plot_analysis = True, labels=None, before_after_label=["", ""]):

    rows, cols = 2, 2
    keys  = ['Reflectance_plot1', 'QualityFactor_plot0', 'Sensitivity_plot0', 'FigureOfMerit_plot0']
    keys2 = [None, 'QualityFactor_plot1', None, None]

    texts = ['(a)', '(b)', '(c)', '(d)', '(e)', '(f)', '(g)', '(h)', '(i)', '(j)', '(k)', '(l)', '(m)', '(n)', '(o)', '(p)', '(q)', '(r)', '(s)', '(t)', '(u)', '(v)', '(w)', '(x)', '(y)', '(z)']
    colors = ['blue', 'red', 'green', 'black', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan', 'magenta', 'yellow', 'lime', 'teal', 'coral', 'lightblue', 'lightgreen', 'lightcoral', 'lightpink', 'lightgray', 'lightolive', 'lightcyan', 'lightmagenta', 'lightyellow', 'lightlime', 'lightteal', 'lightcoral']
    fig, axes = plt.subplots(rows, cols, figsize=(13, 15))
    for i, key in enumerate(keys):
        x_index, y_index = i//2, i%2
        if key == 'Reflectance_plot1':
            data, x, y, _, z, x_label, y_label, z_label = prepare_data(data, key, key2=None, config=config)
            heatmap = axes[x_index, y_index].pcolor(x,y,z, cmap='inferno') # 'hot', 'viridis' , 'inferno' , 'plasma' , 'magma' , 'cividis' , 'twilight' , 'twilight_shifted'
            axes[x_index, y_index].set_xlabel(x_label, fontsize=13)
            axes[x_index, y_index].set_ylabel(y_label, fontsize=13)
            fig.colorbar(heatmap, ax=axes[x_index, y_index], label=z_label)      
        else:
            data, x, y, y2, _, x_label, y_label, _ = prepare_data(data, key, key2=keys2[i], config=config)
            if y is not None:
                axes[x_index, y_index].plot(x, y, color=colors[i])
            if y2 is not None:
                axes[x_index, y_index].plot(x, y2, color=colors[i+(rows*cols)], linestyle='--')
            axes[x_index, y_index].grid()
            axes[x_index, y_index].set_xlabel(x_label, fontsize=13)
            axes[x_index, y_index].set_ylabel(y_label, fontsize=13)

            # y = y if y2 is None else torch.concatenate((y.unsqueeze(-1), y2.unsqueeze(-1)), dim=-1)
            # set_x_and_y_limits(axes[x_index, y_index], x=x, y=y)
            if labels is not None and y is not None and y2 is not None:
                labels_final = [before_after_label[0]+str(label)+before_after_label[1] for label in labels]
                # axes[x_index, y_index].legend(labels_final, loc='best')
        
                axes[x_index, y_index].legend(labels_final, loc='lower left')

            axes[x_index, y_index].text(0.1, 0.85, texts[i], color='black', backgroundcolor='white', transform=axes[x_index, y_index].transAxes, fontsize=20)
    # save the figure with very high resolution
    plt.savefig(experiment_path+'/'+data_name+'_data_.png', dpi=600)

    plt.show()
    plt.close()
# print(data)
    return data


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly
# plotly.offline.init_notebook_mode(connected=True)

def get_data_ploty(plot_data, down_size=100):
    x = plot_data['x'].cpu().numpy()[::down_size]
    y = plot_data['y'].cpu().numpy() # we don't downsample the y values
    z = plot_data['z'].cpu().numpy()[:,::down_size]
    print('x shape:', x.shape, 'y shape:', y.shape, 'z shape:', z.shape)
    return x, y, z



def plot_data_ploty(data, config, experiment_path, data_name, down_size=100, plot_heatmap = True, plot_analysis = True, labels=None, before_after_label=["", ""]):
    if plot_heatmap:
        # Create the first figure for the heatmap
        fig1 = make_subplots(rows=1, cols=2)

        key = 'Reflectance_plot0'
        plot_data = data[key]
        x, y, z = get_data_ploty(plot_data, down_size=down_size)
        colorscale = 'inferno' # 'hot', 'viridis' , 'inferno' , 'plasma' , 'magma' , 'cividis' , 'twilight' , 'twilight_shifted' , 'jet' , 'turbo' , 'thermal' , 'hsv' , 'gray' , 'bone' , 'pink' , 'spring' , 'summer' , 'autumn' , 'winter' , 'cool' , 'Wistia' , 'hot' , 'afmhot' , 'gist_heat' , 'copper'

        heatmap = go.Heatmap(
            x=x,
            y=y,
            z=z,
            colorscale=colorscale
        )

        key = 'Reflectance_plot1'
        plot_data = data[key]
        x, y, z = get_data_ploty(plot_data, down_size=down_size)

        heatmap2 = go.Heatmap(
            x=x,
            y=y,
            z=z,
            colorscale=colorscale
        )

        fig1.add_trace(heatmap, row=1, col=1)
        fig1.add_trace(heatmap2, row=1, col=2)

        fig1.update_layout(
            xaxis_title=plot_data['x_label'],
            yaxis_title=plot_data['y_label'],
            coloraxis_colorbar=dict(title=plot_data['z_label'])
        )

        # Set x and y limits (you'll need to implement set_x_and_y_limits for Plotly)
        # set_x_and_y_limits(fig1, config[key])

        # fig1.write_image(experiment_path + '/' + data_name + "Reflectance.png")
        # fig1.write_html("image.html")
        fig1.show()
        
    if plot_analysis:
        # Create the second figure with subplots
        # All keys for defect : ['band_gap_begining_plot0', 'band_gap_begining_plot1', 'band_gap_begining_data', 'band_gap_end_plot0', 'band_gap_end_plot1', 'band_gap_end_data', 'band_gap_width_plot0', 'band_gap_width_plot1', 'band_gap_width_data', 'band_gap_center_plot0', 'band_gap_center_plot1', 'band_gap_center_data', 'average_peak_intensity_plot0', 'average_peak_intensity_plot1', 'average_peak_intensity_data', 'peak_wavelength_plot0', 'peak_wavelength_plot1', 'peak_wavelength_data', 'position_left_defect_plot0', 'position_left_defect_plot1', 'position_left_defect_data', 'position_right_defect_plot0', 'position_right_defect_plot1', 'position_right_defect_data', 'FullWidthHalfMax_plot0', 'FullWidthHalfMax_plot1', 'FullWidthHalfMax_data', 'defect_intensity_T_plot0', 'defect_intensity_T_plot1', 'defect_intensity_T_data', 'defect_intensity_R_plot0', 'defect_intensity_R_plot1', 'defect_intensity_R_data', 'defect_peak_width_plot0', 'defect_peak_width_plot1', 'defect_peak_width_data', 'QualityFactor_plot0', 'QualityFactor_plot1', 'QualityFactor_data', 'Sensitivity_plot0', 'Sensitivity_data', 'FigureOfMerit_plot0', 'FigureOfMerit_data']
        # All keys for periodic : dict_keys(['band_gap_begining_plot0', 'band_gap_begining_plot1', 'band_gap_begining_data', 'band_gap_end_plot0', 'band_gap_end_plot1', 'band_gap_end_data', 'band_gap_width_plot0', 'band_gap_width_plot1', 'band_gap_width_data', 'band_gap_center_plot0', 'band_gap_center_plot1', 'band_gap_center_data', 'average_peak_intensity_plot0', 'average_peak_intensity_plot1', 'average_peak_intensity_data', 'peak_wavelength_plot0', 'peak_wavelength_plot1', 'peak_wavelength_data', 'position_left_defect_plot0', 'position_left_defect_plot1', 'position_left_defect_data', 'position_right_defect_plot0', 'position_right_defect_plot1', 'position_right_defect_data', 'FullWidthHalfMax_plot0', 'FullWidthHalfMax_plot1', 'FullWidthHalfMax_data', 'defect_intensity_T_plot0', 'defect_intensity_T_plot1', 'defect_intensity_T_data', 'defect_intensity_R_plot0', 'defect_intensity_R_plot1', 'defect_intensity_R_data', 'defect_peak_width_plot0', 'defect_peak_width_plot1', 'defect_peak_width_data', 'QualityFactor_plot0', 'QualityFactor_plot1', 'QualityFactor_data', 'Sensitivity_plot0', 'Sensitivity_data', 'FigureOfMerit_plot0', 'FigureOfMerit_data', 'Reflectance_plot0', 'Reflectance_plot1', 'Reflectance_data'])

        # keys = ['peak_wavelength_plot0', 'FullWidthHalfMax_plot0', 'QualityFactor_plot0', 'Sensitivity_plot0', 'FigureOfMerit_plot0', 'band_gap_begining_plot0', 'band_gap_end_plot0', 'band_gap_width_plot0',   ]
        # keys2 = ['peak_wavelength_plot1', 'FullWidthHalfMax_plot1', 'QualityFactor_plot1', 'Sensitivity_plot0', 'FigureOfMerit_plot0', 'band_gap_begining_plot1', 'band_gap_end_plot1', 'band_gap_width_plot1',  ]
        # rows, cols = 3, 3

        keys = ['band_gap_begining_plot0', 'band_gap_end_plot0', 'band_gap_width_plot0', 'Sensitivity_plot0', 'average_peak_intensity_plot0', 'band_gap_center_plot0',   ]
        keys2 = ['band_gap_begining_plot1', 'band_gap_end_plot1', 'band_gap_width_plot1', 'Sensitivity_plot0', 'average_peak_intensity_plot1', 'band_gap_center_plot0',  ]
        rows, cols = 2, 3
        
        
        texts = ['(a)', '(b)', '(c)', '(d)', '(e)', '(f)', '(g)', '(h)', '(i)', '(j)', '(k)', '(l)', '(m)', '(n)', '(o)', '(p)', '(q)', '(r)', '(s)', '(t)', '(u)', '(v)', '(w)', '(x)', '(y)', '(z)']
        colors = ['blue', 'red', 'green', 'black', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan', 'magenta', 'yellow', 'lime', 'teal', 'coral', 'lightblue', 'lightgreen', 'lightcoral', 'lightpink', 'lightgray', 'lightolive', 'lightcyan', 'lightmagenta', 'lightyellow', 'lightlime', 'lightteal', 'lightcoral']

        fig2 = make_subplots(rows=rows, cols=cols, subplot_titles=texts)

        additonal_word = '_plot'

        for i, key in enumerate(keys):
            plot_data = data[key]
            plot_data2 = data[keys2[i]]
            row, col = i // 3 + 1, i % 3 + 1

            x = plot_data['x'].cpu().numpy()
            y = plot_data['y'].cpu().numpy()

            x2 = plot_data2['x'].cpu().numpy()
            y2 = plot_data2['y'].cpu().numpy()

            trace = go.Scatter(x=x, y=y, mode='lines', line=dict(color=colors[i]),
                                name=(before_after_label[0]+str(labels[0])+before_after_label[1]) if labels is not None else None)
            trace2 = go.Scatter(x=x2, y=y2, mode='lines', line=dict(color=colors[i], dash='dash'),
                                
                                 name=(before_after_label[0]+str(labels[1])+before_after_label[1]) if labels is not None else None)

            fig2.add_trace(trace, row=row, col=col)
            fig2.add_trace(trace2, row=row, col=col)

            fig2.update_xaxes(title_text=plot_data['x_label'], row=row, col=col)
            fig2.update_yaxes(title_text=plot_data['y_label'], row=row, col=col)
            
            # Set x and y limits (you'll need to implement set_x_and_y_limits for Plotly)
            # set_x_and_y_limits(fig2, config[key], x=x, y=y, row=row, col=col)

        fig2.update_layout(height=900, width=800, showlegend=False)

        # fig2.write_image(experiment_path + '/' + data_name + '_data_.png')
        # fig2.write_html("image.html")


        fig2.show()
    return data


## Single data analysis

#### Finding the defect peaks and their properites

In [ ]:
def limit_x_y( x, y, x_value_limit, reversed=False):
    condition = (x>x_value_limit[0]) & (x<x_value_limit[1])
    if reversed:
        condition = ~condition
    x_index = torch.where(condition)
    x = x[x_index]
    y = y[x_index]
    print('x shape:', x.shape, 'y shape:', y.shape)
    return x, y, x_index


def get_defect_peak_and_properiety(data,  plot_word='_plot0', tol=5, x_limit=None, x_limit_correct=None, filled_values = None,):

    assert x_limit_correct is not None or filled_values is not None, 'Either x_limit_correct or filled_values should be provided'

    x = data['band_gap_begining'+plot_word]['x']
    PBG_range = torch.cat([data['band_gap_begining'+plot_word]['y'].unsqueeze(-1), data['band_gap_end'+plot_word]['y'].unsqueeze(-1)], dim=-1)
    x_limit = [x[0], x[-1]] if x_limit is None else x_limit
    if x_limit_correct is not None:
        print('x shape:', x.shape, 'PBG_range shape:', PBG_range.shape)
        x, PBG_range, _ = limit_x_y(x,PBG_range,x_limit)
        plot_y_ploty(x=x, y=PBG_range.T, labels=['begining', 'end'])
        x_correct, PBG_range_correct, _ = limit_x_y(x,PBG_range,x_limit_correct)


        # fit the correct data to a line
        from scipy.optimize import curve_fit
        def linear(x, m, c):
            return m*x + c
        popt, _ = curve_fit(linear, x_correct, PBG_range_correct[:,0])
        popt_2, _ = curve_fit(linear, x_correct, PBG_range_correct[:,1])
        print('popt:', popt, 'popt_2:', popt_2)


        # replace the values with the fitted values
        x_to_corrected, _, x_index = limit_x_y(x,PBG_range,x_limit_correct, reversed=True)
        print('x_to_corrected shape:', x_to_corrected.shape)
        PBG_range[x_index] = torch.cat([linear(x_to_corrected, *popt).unsqueeze(-1), linear(x_to_corrected, *popt_2).unsqueeze(-1)], dim=-1)
        plot_y_ploty(x=x, y=PBG_range.T, labels=['begining', 'end'])

    elif filled_values is not None:
        tol_range = 0.01
        range_start = x[0] - tol_range
        for filled_value in filled_values:
            single_x         = [range_start,filled_value[0][0]]
            single_PBG_range = [filled_value[1][0], filled_value[1][1]]
            _,_, x_index = limit_x_y(x,PBG_range,single_x)
            PBG_range[x_index] = torch.ones_like(PBG_range[x_index]) * torch.tensor(single_PBG_range).unsqueeze(0)
            range_start = filled_value[0][0] - tol_range
    else: 
        raise ValueError('Either x_limit_correct or filled_values should be provided')

    
    R = data['Reflectance'+plot_word]['z']
    wavelength = data['Reflectance'+plot_word]['x']
    variable   = data['Reflectance'+plot_word]['y']
    gamma_ray_doses = [0,70] #Gy
    refractive_index_variable_material= refractive_index_PVA(wavelength, gamma_ray_doses)
    variable, R, _ = limit_x_y(variable,R,x_limit)


    print(R.shape, variable.shape, wavelength.shape)
    R_PBG = torch.ones_like(R)
    for i in range(R.shape[0]):
        wavelength_PBG_indices = torch.where((wavelength>PBG_range[i,0]+tol) & (wavelength<PBG_range[i,1]-tol))
        R_temp_PBG = R_PBG[i].squeeze()
        R_temp_R   = R[i].squeeze()
        R_temp_PBG[wavelength_PBG_indices] = R_temp_R[wavelength_PBG_indices]
        R_PBG[i] = R_temp_PBG


    fig = make_subplots(rows=1, cols=1)
    # z =R_PBG[:,::100]
    z =R_PBG[:,::5]
    print(z.shape)
    heatmap = go.Heatmap(
        z=z,
        colorscale='Plasma'
    )
    fig.add_trace(heatmap, row=1, col=1)
    fig.show()

    
    import scipy
    import numpy as np
    wavelength_peak  = np.zeros(R_PBG.shape[0])
    FullWidthHalfMax = np.zeros(R_PBG.shape[0])
    Qualityfactor    = np.zeros(R_PBG.shape[0])
    defect_intensity_T = np.zeros(R_PBG.shape[0])
    defect_intensity_R = np.zeros(R_PBG.shape[0])
    refractive_index_wavelength_peak = np.zeros(R_PBG.shape[0])
    for i in range(R_PBG.shape[0]):
        try:
            peaks, peaks_properties = scipy.signal.find_peaks(1-R_PBG[i].cpu().numpy().squeeze(), prominence = 0.1)
            wavelength_peak[i] = wavelength[peaks[peaks_properties['prominences'].argsort()[-1]]]
            peak_widths =  scipy.signal.peak_widths(1-R_PBG[i].cpu().numpy().squeeze(), peaks, rel_height=0.5)
            # FullWidthHalfMax[i] = peak_widths[0][peaks_properties['prominences'].argsort()[-1]]
            index_peak_fwhm = peaks_properties['prominences'].argsort()[-1]
            peak_FWHM, left_index, right_index = peak_widths[0][index_peak_fwhm], peak_widths[2][index_peak_fwhm], peak_widths[3][index_peak_fwhm]
            
            FullWidthHalfMax[i] = peak_FWHM * (wavelength[-1]-wavelength[0]) / len(wavelength)
            
            Qualityfactor[i] = wavelength_peak[i]/FullWidthHalfMax[i]

            defect_intensity_T[i] = 1-R_PBG[i][peaks[peaks_properties['prominences'].argsort()[-1]]]
            defect_intensity_R[i] = R_PBG[i][peaks[peaks_properties['prominences'].argsort()[-1]]]
            plot_number = int (plot_word[-1])
            refractive_index_wavelength_peak[i] = refractive_index_variable_material[plot_number, torch.where(wavelength==wavelength_peak[i])].item()
        except:
            print('Error in finding the peak')
            wavelength_peak[i] = np.nan
            FullWidthHalfMax[i] = np.nan
            Qualityfactor[i] = np.nan
            defect_intensity_T[i] = np.nan
            defect_intensity_R[i] = np.nan
            refractive_index_wavelength_peak[i] = np.nan            
    print(wavelength_peak.shape)
    print(variable.shape)

    data['peak_wavelength'+plot_word] = {'x':variable, 'y':torch.tensor(wavelength_peak), 'x_label':data['peak_wavelength'+plot_word]['x_label'], 'y_label':data['peak_wavelength'+plot_word]['y_label']}
    data['FullWidthHalfMax'+plot_word] = {'x':variable, 'y':torch.tensor(FullWidthHalfMax), 'x_label':data['FullWidthHalfMax'+plot_word]['x_label'], 'y_label':data['FullWidthHalfMax'+plot_word]['y_label']}
    data['QualityFactor'+plot_word] = {'x':variable, 'y':torch.tensor(Qualityfactor), 'x_label':data['QualityFactor'+plot_word]['x_label'], 'y_label':data['QualityFactor'+plot_word]['y_label']}
    data['defect_intensity_T'+plot_word] = {'x':variable, 'y':torch.tensor(defect_intensity_T), 'x_label':data['defect_intensity_T'+plot_word]['x_label'], 'y_label':data['defect_intensity_T'+plot_word]['y_label']}
    data['defect_intensity_R'+plot_word] = {'x':variable, 'y':torch.tensor(defect_intensity_R), 'x_label':data['defect_intensity_R'+plot_word]['x_label'], 'y_label':data['defect_intensity_R'+plot_word]['y_label']}
    data['refractive_index_wavelength_peak'+plot_word] = {'x':variable, 'y':torch.tensor(refractive_index_wavelength_peak), 'x_label':"", 'y_label':""}
    return data

In [ ]:
plot_words = ['_plot0','_plot1']
tols = [0, 0]
x_limits = [[40,90],[40,90]]
x_limit_corrects = [[40,90],[40,90]]
# # [[[range1],[start,end],...]...]
# from_to_values = [
#     [[[0.4],[530,560]],[[0.5],[525,560]],[[0.6],[520,560]],[[0.71],[515,560]]],
#     [[[0.4],[557,580]],[[0.57],[547,580]],[[0.71],[535,580]]],
#                   ]
for i in range(len(plot_words)):
    data = get_defect_peak_and_properiety(data, plot_word=plot_words[i], tol=tols[i], x_limit=x_limits[i], x_limit_correct=x_limit_corrects[i])
plot_data_ploty(data, config, experiment_path, data_name, labels=[0,70], before_after_label=['Dose = ', ' Gy'], down_size=1) #, plot_heatmap = False)
    
    # data = get_defect_peak_and_properiety(data, plot_word=plot_words[i], filled_values=from_to_values[i])

In [ ]:
data["Sensitivity_plot0"] = {'x':data['peak_wavelength_plot0']['x'], 
                             'y':(data['peak_wavelength_plot1']['y']-data['peak_wavelength_plot0']['y'])/(data['refractive_index_wavelength_peak_plot1']['y']-data['refractive_index_wavelength_peak_plot0']['y']),
                             'x_label':data['Sensitivity_plot0']['x_label'], 
                             'y_label':data['Sensitivity_plot0']['y_label']}
data["FigureOfMerit_plot0"] = {'x':data['peak_wavelength_plot0']['x'],'y':data['Sensitivity_plot0']['y']/data['FullWidthHalfMax_plot1']['y'],
                                 'x_label':data['FigureOfMerit_plot0']['x_label'], 'y_label':data['FigureOfMerit_plot0']['y_label']}
print(data['Sensitivity_plot0']['y'])

plot_y_ploty(x=data['Sensitivity_plot0']['x'], y=data['Sensitivity_plot0']['y'],save_path="Sensitivity_plot0")
plot_y_ploty(x=data['FigureOfMerit_plot0']['x'], y=data['FigureOfMerit_plot0']['y'],save_path="FigureOfMerit_plot0")
torch.save(data, experiment_path+'data/'+data_name+'_analysis2'+'.pt')




# ALL the data 

In [ ]:
import yaml
import pandas as pd
all_data = ['layer_1_thickness', 'layer_2_thickness', 'porosity_1', 'porosity_2',]# 'rho0', 'N', 'layer_3_thickness', 'defect_thickness', ]

# all_data = ['porosity_2']
save_data_as_excel = True
save_data_as_excel = False

for data_name in all_data:


    # experiment_path = '/mnt/BTNAS003/Users/ahmed/repos/nerfstudio_ayman/materials/photonics/Experiments/GammaRay/'
    # experiment_path = '/mnt/BTNAS003/Users/ahmed/repos/nerfstudio_ayman/materials/photonics/Experiments/GammaRay_tri/'
    # experiment_path = '/Users/ayman/Documents/photonics/Experiments/GammaRay_tri/'
    # experiment_path = '/Users/ayman/Documents/photonics/Experiments/GammaRay_mirror/'
    experiment_path = '/mnt/BTNAS003/Users/ahmed/repos/nerfstudio_ayman/materials/photonics/Experiments/GammaRay_mirror/'
    # experiment_path = '/Users/ayman/Documents/photonics/Experiments/E_coli/'
    # experiment_path = '/mnt/BTNAS003/Users/ahmed/repos/nerfstudio_ayman/materials/photonics/Experiments/E_coli/'



    try: 
        data ={**torch.load(experiment_path+'data/'+data_name+'_analysis2'+'.pt'),**torch.load(experiment_path+'data/'+data_name+'.pt')}
    except:
        data ={**torch.load(experiment_path+'data/'+data_name+'_analysis'+'.pt'),**torch.load(experiment_path+'data/'+data_name+'.pt')}

    print(data.keys())  

    try:
        config = {**yaml.load(open(experiment_path+'/config/' + data_name + '.yaml', 'r'), Loader=yaml.FullLoader),
                  **yaml.load(open(experiment_path+'/config/' + data_name +'_analysis' + '.yaml', 'r'), Loader=yaml.FullLoader)}
        # config = None
    except:
        config = None

    data = plot_data_matplotlib(data, config, experiment_path, data_name, labels=[0.0,0.5], before_after_label=['Volume fraction = ', ' '])#, plot_heatmap = False)
    # data = plot_data_ploty(data, config, experiment_path, data_name, labels=[0,70], before_after_label=['Dose = ', ' Gy'], down_size=100) #, plot_heatmap = False)



    if save_data_as_excel:
        # first remove keep only the data with the following keys
        # keys = ['peak_wavelength_plot1', 'defect_intensity_R_plot1', 'FullWidthHalfMax_plot1', 'QualityFactor_plot1', 'Sensitivity_plot0', 'FigureOfMerit_plot0']
        keys = ['band_gap_begining_plot0', 'band_gap_end_plot0', 'band_gap_width_plot0', 'Sensitivity_plot0', 'average_peak_intensity_plot0', 'band_gap_center_plot0',   ]

        data = {key: data[key] for key in keys}
        dataframe_dict = {data[keys[0]]['x_label']: data[keys[0]]['x'].cpu().numpy(),**{data[key]['y_label']: data[key]['y'].cpu().numpy() for key in keys}}  # First column is the x values and the rest are the y values
        print([dataframe_dict[key].shape for key in dataframe_dict.keys()])
        df = pd.DataFrame(dataframe_dict)

        if config is not None:
            xlim = config[keys[0]]['xlim']
        else:
            xlim = None

        last_data_point = df.iloc[-1,:]
        # if xlim is not None:
        #     df = df[(df[data[keys[0]]['x_label']] > xlim[0]) & (df[data[keys[0]]['x_label']] <= xlim[1])]    # remove all the data that is outside the xlim

        # if xlim is None: 
        # df = df.iloc[::10, :]  # reduce the number of points save only one point for every 10 points
        df = df.iloc[::5, :]  # reduce the number of points save only one point for every 50 points
        df = pd.concat([df, last_data_point.to_frame().T])

        excel_path = os.path.join(experiment_path, 'excel')
        os.makedirs(excel_path, exist_ok=True)
        df.to_excel(excel_path + '/' + data_name + '.xlsx', index=False)

